In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from copy import deepcopy

import time
from datetime import datetime



import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [2]:

USE_CUDA = torch.cuda.is_available()
# USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


# Iris Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [3]:
my_random_seed = 42
random.seed(my_random_seed)

In [4]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [5]:
iris = datasets.load_iris()
xs_raw = iris.data[:, :]
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = iris.target
ys_onehot = one_hot_encode(ys)

In [6]:
xs = torch.from_numpy(xs).to(device)
ys = torch.from_numpy(ys).to(device)

In [7]:
n_inputs = xs.shape[1]
n_outputs = ys_onehot.shape[1]
(n_inputs, n_outputs)

(4, 3)

Let's have a look at the data we are working with

In [8]:
xs[:5]

tensor([[-0.9007,  1.0190, -1.3402, -1.3154],
        [-1.1430, -0.1320, -1.3402, -1.3154],
        [-1.3854,  0.3284, -1.3971, -1.3154],
        [-1.5065,  0.0982, -1.2834, -1.3154],
        [-1.0218,  1.2492, -1.3402, -1.3154]], device='cuda:1',
       dtype=torch.float64)

In [ ]:
ys[:5]

tensor([0, 0, 0, 0, 0], device='cuda:1')

In [ ]:
ys_onehot[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `CrossEntropyLoss` from `PyTorch`

In [ ]:
def eval_genomes(genomes, config):
    loss = nn.CrossEntropyLoss()
    loss = loss.to(device)

    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(ys)))

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [ ]:
config_path = "./config-iris"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


In [ ]:
base_config.pop_size

2

We also want to put a hard limit on how long this can go on for.

In [ ]:
maxNGenerations = 20

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [ ]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.CrossEntropyLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment 1: Vary population size

The first experiment is going to examine the difference in run time different population sizes. 

In [ ]:
population_points = [2, 5, 10, 25, 50, 100]

In [ ]:
base_config.pop_size

2

In [ ]:
saveLocationTemplate = './../../data/experiments/iris/experiment-populationgpu-{}-{}/'

## Start the experiment

In [ ]:
for pop_size in population_points:
    for iteration_no in range(5):
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        print("################################################")
        print("################################################")
        print("Starting population {} iteration {}".format(pop_size, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
        
        
        config = deepcopy(base_config)
        config.pop_size = pop_size
        
        saveLocation = saveLocationTemplate.format(pop_size, iteration_no)
        
        p = instantiate_population(config, xs, ys, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(eval_genomes, maxNGenerations)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(xs, ys):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))
        
        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }
        
        
        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
            data = (p, g, ancestry, ancestors, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

    
#         visualize.create_ancestry_video(p.config, 
#                                         g, 
#                                         ancestry, 
#                                         ancestors, 
#                                         p.reporters.reporters[1], 
#                                         pathname=saveLocation)
        print("################################################")
        print("################################################")
        print("Have finished population {} iteration {}".format(pop_size, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("The time is {}".format(end_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
    

################################################
################################################
Starting population 2 iteration 0
Started at 07/11/2019, 03:49:08
################################################
################################################

 ****** Running generation 0 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.9145, grad_fn=<DivBackward0>)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Population's average fitness: 1.01112 stdev: 0.08233
Best fitness: 1.09345 - size: (3, 12) - species 2 - id 2
ending generation %s
Average adjusted fitness: 0.082
Mean genetic distance 2.091, standard deviation 2.051
Population of 4 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     2      0.9    0.000     0
     2    0     2      1.1    0.165     0
Total extinctions: 0
Generation time: 34.499 sec

 ****** Running generation 1 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.8864, grad_fn=<DivBackward0>)
Population's average fitness: 1.01979 stdev: 0.09182
Best fitness: 1.12811 - size: (3, 11) - species 2 - id 4


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7f8cc8517240>, 'fitness': 1.1281148195266724, 'firstDerivatives': [0.0, 0.0346604585647583], 'secondDerivatives': [0.0, 0.0346604585647583]}
Key: 4
Fitness: 1.12811481952

Population's average fitness: 1.01979 stdev: 0.09182
Best fitness: 1.12811 - size: (3, 11) - species 2 - id 4
ending generation %s
Average adjusted fitness: 0.091
Mean genetic distance 2.091, standard deviation 2.051
Population of 4 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    8     2      0.9    0.000     8
     2    8     2      1.1    0.182     7
Total extinctions: 0
Generation time: 69.800 sec (65.026 average)

 ****** Running generation 9 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.8864, grad_fn=<DivBackward0>)
Population's average fitness: 1.01979 stdev: 0.09182
Best fitness: 1.12811 - size: (3, 11) - species 2 - id 4
ending generation %s
Average adjusted fitness: 0.091
Mean genetic distance 2.091, standard deviation 2.051
Population of 4 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  

gen is 19
previous generation is [4]
skey is 4
gen is 18
previous generation is [4]
skey is 4
gen is 17
previous generation is [4]
skey is 4
gen is 16
previous generation is [4]
skey is 4
gen is 15
previous generation is [4]
skey is 4
gen is 14
previous generation is [4]
skey is 4
gen is 13
previous generation is [4]
skey is 4
gen is 12
previous generation is [4]
skey is 4
gen is 11
previous generation is [4]
skey is 4
gen is 10
previous generation is [4]
skey is 4
gen is 9
previous generation is [4]
skey is 4
gen is 8
previous generation is [4]
skey is 4
gen is 7
previous generation is [4]
skey is 4
gen is 6
previous generation is [4]
skey is 4
gen is 5
previous generation is [4]
skey is 4
gen is 4
previous generation is [4]
skey is 4
gen is 3
previous generation is [4]
skey is 4
gen is 2
previous generation is [4]
skey is 4
gen is 1
previous generation is [4]
skey is 4
gen is 0
previous generation is [2, 4]
skey is 2
skey is 4
have calculated the ancestry
Saving checkpoint to fullSta

...

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.8405, grad_fn=<DivBackward0>)
Population's average fitness: 0.90994 stdev: 0.18244
Best fitness: 1.18977 - size: (3, 12) - species 2 - id 3
ending generation %s
Average adjusted fitness: 0.224
Mean genetic distance 2.483, standard deviation 1.557
Population of 6 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   10     2      1.0    0.240     9
     2   10     2      1.2    0.433    10
     3   10     2      0.7    0.000    10
Total extinctions: 0
Generation time: 98.411 sec (96.894 average)

 ****** Running generation 11 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.8405, grad_fn=<DivBackward0>)
Population's average fitness: 0.90994 stdev: 0.18244
Best fitness: 1.18977 - size: (3, 12) - species 2 - id 3
ending generation %s
Average adjusted fitness: 0.224
Mean genetic dis

gen is 19
previous generation is [3]
skey is 3
gen is 18
previous generation is [3]
skey is 3
gen is 17
previous generation is [3]
skey is 3
gen is 16
previous generation is [3]
skey is 3
gen is 15
previous generation is [3]
skey is 3
gen is 14
previous generation is [3]
skey is 3
gen is 13
previous generation is [3]
skey is 3
gen is 12
previous generation is [3]
skey is 3
gen is 11
previous generation is [3]
skey is 3
gen is 10
previous generation is [3]
skey is 3
gen is 9
previous generation is [3]
skey is 3
gen is 8
previous generation is [3]
skey is 3
gen is 7
previous generation is [3]
skey is 3
gen is 6
previous generation is [3]
skey is 3
gen is 5
previous generation is [3]
skey is 3
gen is 4
previous generation is [3]
skey is 3
gen is 3
previous generation is [3]
skey is 3
gen is 2
previous generation is [3]
skey is 3
gen is 1
previous generation is [3]
skey is 3
gen is 0
previous generation is [3]
skey is 3
have calculated the ancestry
Saving checkpoint to fullStatus.xplnt
###

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7627, grad_fn=<DivBackward0>)
Population's average fitness: 1.11609 stdev: 0.21559
Best fitness: 1.31107 - size: (3, 12) - species 2 - id 5
ending generation %s
Average adjusted fitness: 0.255
Mean genetic distance 1.830, standard deviation 1.727
Population of 5 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   10     2      0.9    0.038    10
     2   10     3      1.3    0.473    10
Total extinctions: 0
Generation time: 83.432 sec (80.017 average)

 ****** Running generation 11 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7518, grad_fn=<DivBackward0>)
Population's average fitness: 1.13254 stdev: 0.22781
Best fitness: 1.33010 - size: (3, 10) - species 2 - id 17


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7f8cc849ae10>, 'fitness': 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7402, grad_fn=<DivBackward0>)
Population's average fitness: 1.13062 stdev: 0.22807
Best fitness: 1.35095 - size: (3, 9) - species 2 - id 18
ending generation %s
Average adjusted fitness: 0.268
Mean genetic distance 2.019, standard deviation 1.728
Population of 5 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   14     2      0.9    0.038    14
     2   14     3      1.4    0.497     2
Total extinctions: 0
Generation time: 74.988 sec (80.088 average)

 ****** Running generation 15 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7402, grad_fn=<DivBackward0>)
Population's average fitness: 1.14292 stdev: 0.23631
Best fitness: 1.35095 - size: (3, 9) - species 2 - id 18
ending generation %s
Average adjusted fitness: 0.278
Mean genetic distance 1.935, standard deviation 1.718
Popu

gen is 19
previous generation is [23]
skey is 23
gen is 18
previous generation is [23]
skey is 23
gen is 17
previous generation is [23]
skey is 23
gen is 16
previous generation is [18, 23]
skey is 18
skey is 23
gen is 15
previous generation is [18]
skey is 18
gen is 14
previous generation is [18]
skey is 18
gen is 13
previous generation is [18]
skey is 18
gen is 12
previous generation is [18]
skey is 18
gen is 11
previous generation is [17, 18, 5]
skey is 17
skey is 18
skey is 5
gen is 10
previous generation is [17, 5, 15]
skey is 17
skey is 5
skey is 15
gen is 9
previous generation is [5, 15]
skey is 5
skey is 15
gen is 8
previous generation is [5, 15]
skey is 5
skey is 15
gen is 7
previous generation is [5]
skey is 5
gen is 6
previous generation is [5]
skey is 5
gen is 5
previous generation is [5]
skey is 5
gen is 4
previous generation is [5]
skey is 5
gen is 3
previous generation is [5]
skey is 5
gen is 2
previous generation is [5]
skey is 5
gen is 1
previous generation is [5]
skey 